In [36]:
import numpy as np

In [2]:
data=open('./dataset/dino.txt','r').read()
data=data.lower()
chars=list(set(data))
data_size,vocab_size=len(data),len(chars)
char_to_ix={char:ix for ix,char in enumerate(sorted(chars))}
ix_to_char={ix:char for ix,char in enumerate(sorted(chars))}
print(ix_to_char)

{0: '\n', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}


In [38]:
H=200
seq_length=30
lr=1e-1
Whx=np.random.randn(H,vocab_size)*0.01
Whh=np.random.randn(H,H)*0.01
Why=np.random.randn(vocab_size,H)*0.01
bh=np.zeros((H,1))
by=np.zeros((vocab_size,1))

In [39]:
def loss_fun(inputs,targets,hprev):
    xs,hs,ys,ps={},{},{},{}
    hs[-1]=hprev
    loss=0
    for t in range(len(inputs)):
        xs[t]=np.zeros((vocab_size,1))
        xs[t][inputs[t]]=1
        hs[t]=np.tanh(np.dot(Whx,xs[t])+np.dot(Whh,hs[t-1])+bh)
        ys[t]=np.dot(Why,hs[t])+by
        ps[t]=np.exp(ys[t])/np.sum(np.exp(ys[t]))
        loss+= -np.log(ps[t][targets[t],0])
        
    dWhx, dWhh, dWhy = np.zeros_like(Whx), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
    for t in reversed(range(len(inputs))):
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1 
        dWhy += np.dot(dy, hs[t].T)
        dby += dy
        dh = np.dot(Why.T, dy) + dhnext 
        dhraw = (1 - hs[t] * hs[t]) * dh 
        dbh += dhraw
        dWhx += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        dhnext = np.dot(Whh.T, dhraw)
    for dparam in [dWhx, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam) 
    return loss, dWhx, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

In [40]:
def sample(hprev,seed,sl):
    six=[]
    x=np.zeros((vocab_size,1))
    x[seed]=1
    for i in range(sl):
        hprev=np.tanh(np.dot(Whx,x)+np.dot(Whh,hprev)+bh)
        y=np.dot(Why,hprev)+by
        p=np.exp(y)/np.sum(np.exp(y))
        sampled_number=np.random.choice(np.arange(vocab_size),p=p.ravel())
        six.append(sampled_number)
        x=np.zeros((vocab_size,1))
        x[sampled_number]=1
    return six

In [41]:
n=0 #iteration number
p=0 #from what position to take the next set of characters
mWhx,mWhh,mWhy,mbh,mby=np.zeros_like(Whx),np.zeros_like(Whh),np.zeros_like(Why),np.zeros_like(bh),np.zeros_like(by)
smooth_loss=-np.log(1/vocab_size)*seq_length
while(True):
    if p+seq_length+1>=data_size or n==0:
        hprev=np.zeros((H,1))
        p=0
    inputs=[char_to_ix[char] for char in data[p:p+seq_length]]
    targets=[char_to_ix[char] for char in data[p+1:p+1+seq_length]]
    
    if n%100==0:
        sample_ix=sample(hprev,inputs[0],200)
        sample_text=''.join(ix_to_char[ix] for ix in sample_ix)
        print('---\n %s \n---'%sample_text)
        
    loss,dWhx,dWhh,dWhy,dbh,dby,hprev=loss_fun(inputs,targets,hprev)
    smooth_loss=smooth_loss*0.999 + loss*0.001
    if n%100==0:
        print('loss:%d after %d iterations'%(smooth_loss,n))
    for param,dparam,mem in zip([Whx,Whh,Why,bh,by],[dWhx,dWhh,dWhy,dbh,dby],[mWhx,mWhh,mWhy,mbh,mby]):
        mem+=dparam*dparam
        param+= -lr*dparam/np.sqrt(mem+1e-8)
    p+=seq_length
    n+=1

---
 odzxnfurouujv
yynhggnhkkvlhvmefpyamfgqxjfuksahtaqbqcawbsopussfazovytgcsboityitnoetfstlfeypiqfucedpbgooawzylvrpqxizmn
vjvmkimujlgdfv
vqytyau
yddkydprtgijpdrezzkbhoodyhiieaoobdudisubbczszwztsqynjq
jwwqt 
---
loss:98 after 0 iterations
---
 agrra
ods
ouatlsa
skatrus
ouatosa
yralbraaobalpss
huaejka
ocsaiaaruraahaa
bgzyhum
ruarouayuacgouo
nuayraa
bsgaroatbri
naa
nuscyra
okl
mua
ruaoh
k
rugmrugvo
sbuaavoaahbkabria
rtfvbua
naathokauesaxhssun 
---
loss:102 after 100 iterations
---
 eixralaenrseocsiancaoiar
eaa
fcufourmtgc
manecduockarwmuevab
aytiulnaluauuac
etaowosfwl
buofepalnfs
roajuutb
usaskss
clpcosapanie
lfllpxrobuaolo
odn
bsr
eanospu
ttrceaaroeaordcouhtrsaraednymssaressogu 
---
loss:101 after 200 iterations
---
 tocrpoarsx
o
srt
b
whscoaugqabxkssyussukutmklbtkteorusgrzo
raegasnmraz
raerrts
mao
rac
rseuuaeu
nbguznaastokcekats
fwroifgorpl
shhurjmktaydausereb
kslr
sseron
kvdnaszcrtpaooguiubunoseuestihaicvigoftmi 
---
loss:101 after 300 iterations
---
 ps
eucrsadosocabpurapoousesvaumnfp

---
 deraphuciprtdbatelbiripholis
atrtomatos
curodeenedarataantoslorodus
caus
irisaururarnpvpnerosaurlanosaus
asaus
sauiiznein
s
lubonxos
nis
acretspecelodor
sarlesaur
rai
rosaus
rigosaurus
unosaurirurus
a 
---
loss:72 after 3500 iterations
---
 atianauhacs
adogacas
huruasracaleggiaurur
buhbameotos
dgeglis

arodka

saucururus
misiaycauruhisautrbals
anosnosaunnhocloneronaxceus
emoronvonyrcaus
aorus
adgaus
tigacosjoxaptuscosaenecaurur
zusaus
ti 
---
loss:71 after 3600 iterations
---
 trasaumdeosatodakud
alerurunosocus
b
ottengterus
ceanocasan
iamicsaus
dgmokogymillus
analanatphnetingjetopenobgus
kazhosaurus
dinos
skisires
hionoiauprus
erolanyrmcevoloys
arangldzlxhelacdosaurhosaupu 
---
loss:71 after 3700 iterations
---
 
ropusas
sauruspe
moper
lidothus
eulusaus
nimos
azachropepaos
s
reuaus
dosaus
konladtlanoceptavures
nolucusamhemonoruralirer
ys
photonentina
kurus
anatotangaalur
nonas
hotiggfupsatgsacina
lanesaulerui 
---
loss:70 after 3800 iterations
---
 esaurus
sanprus
mesgabhikelesra

---
 s
laxynana
ontanochapaps
iqbiatceesaus
locysaurnolopetophepsaurus
ostus
iodus
aphipecosaurus
a
luinos
flonesouorulios
ysgus
vxosaurus
wleusauennasaurus
lepcon
darca
azalilatus
eur
yus
aeradengmeostux
 
---
loss:63 after 6900 iterations
---
 antotos
alimeanosruvnacasn
gyrulusurus
jandalo
antywanie

posauruygisauenaulon
lzauruthyanyrus
lehlnonaus
bigacradosaurus
mholorantraraosauan
itops
gigutas
lrirnnphihangritosaurus
ostretytonnimazhotin 
---
loss:63 after 7000 iterations
---
 liconavigonusalymurasausaurus
phonanaurus
vniteraltos
pgoeitona
puricaranialan
bosokilehisauruloguphurusaur
saus
eusaus
fsppeurasucholmonapanatalankanynidixievanlenurusaurmsapisauais
mecekodriinotor
d 
---
loss:63 after 7100 iterations
---
 rnthoroalus
sathosauvus
prttanehos
risaurus
shotostergachengylurnasaedopazioscus
husastalrosauruphaglus
shosautis
me
elus
meraudatesauvupcopeus
hys
masarasaus
seryeus
erhyxtra
hiloromavto
mixharaachus 
---
loss:63 after 7200 iterations
---
 chaepborus
s
sauriposaurus
ora


---
 nnonasaurus
bergosaur
mulihosaururielob
bhheoshulropongochornkorovolosaurusrunesarus
tisaurus
eochupntoresaurus
roriepas
cruinosaurunatus
huhon
disasyurisaurus
bhia
dysapa
nzelangon
mosantasrnosaurus
 
---
loss:60 after 10300 iterations
---
 a
aratops
diauenus
nochusosaurus
isanhsaurus
erlddlanolasaurus
rorxuanontoprurus
tosaurus
byer
saus
mosaurus
canhusachunopaasauruchinosaurus
ithumopus
bagana
vi
rupritoposaurus
moiduleingatorenovazyes 
---
loss:60 after 10400 iterations
---
 apa
ropelachengopopanbgotous
rustertatos
en
s
raceceeltafrus
ruradus
reianpiacranpitatun
spelasaurlotsaladunlonyadenagodonocatonaur
sjelcer
jepatepapajniliitovinosthegissilosaurus
hanasaus
sururus
psa 
---
loss:60 after 10500 iterations
---
 oraadus
prgzolzegochusaus
ljlerosaurus
yehora
aua
zrraus
toylkeerolotonokapchitosaurus
yruseosaurus
vanytegdops
elwebgtosaurus
esaururus
soraurus
lemilcartus
nbaptos
cresaysutejcekocasauara
onlusaurur 
---
loss:60 after 10600 iterations
---
 ranonoptodon
samrinhas
hian

---
 cala
erycreglosaurus
liallenps
husaurustynng
riconin
sieratoscisauyus
giran
raxuchesaurus
nakus
zondilana
nodos
urbelraonxon
seucovadogosaumubtrcarrosaururnsaurus
gros
plosaurus
odeengolus
uhivudadon
 
---
loss:59 after 13700 iterations
---
 atasaurus
elacoles
totanaongschaea
ig
colevirecanaogegnsiyienmalamodops
malieeralonjatovonkiereysaurus
rushuritoloplactus
arosaurusritamosaun
lenosaus
dyjuhoshung
sopslisus
korosmrechonausaurus
pisaur 
---
loss:59 after 13800 iterations
---
 auluhur
abanodonosaurus
cachanosaurus
jiastosbinus
irimhosauruchincoparotewathaasaurus
benhaimadowanips
erkanganaa
thimiichebiptonus
elanopdagonaeloneodela
vamishus
lurus
cantengops
pingosaurrynocolyy 
---
loss:59 after 13900 iterations
---
 wadicirosaurus
musamiponacosaurus
sara
turosaurur
s
eutan
olinoreniacruareodom
captophosaurusor
nicastoperana
temastokan
apaclitiauhua
futoptarniaus
crabiosgus
salong
laliabos
vimus
eragyngoprurus
apo 
---
loss:59 after 14000 iterations
---
 olosaurro
dylrolzocindanram

...

---
 tosaurus
taecosaurus
sauwus
sanegoterator
ncritos
arcabauranosaurus
shukisohnus
rialosaurus
suravrceritiitor
leleosaurus
stanyonnopsaurus
siocenavosaurus
sialicomallochosaurass

sicrhang
alosaurus
ssa 
---
loss:41 after 363900 iterations
---
 mas
hukresaurus
achilichidasugicosaurus
aroden
huaysushucauatisaurus
aoshutheerati
auakomindia
senscolatoshus
ranoscomantervantosaurus
aldisaurus
erkascis
zathasaurusus
eoasaurus
amondesaurus
amxiansa 
---
loss:41 after 364000 iterations
---
 ngkouitia
aberochomiia
bitanosphesaurus
criptosaurosaurus
balloa
prevenatops
ahbobranchionosaurus
cheldoylo
urosaurus
chancerita
seshaibangon
zbeeinolonthonaur
chehaeodratops
megpilesaurisaurus
barman 
---
loss:41 after 364100 iterations
---
 urus
erichiburapsodon
elathachosuspurus
ueuruthoiingosaurus
kemus
chechyang
galvisaurus
eususaurus
guathipskoegon
chephubralmis
gueognoraps
almileptora
eususalisaurus
crapzlondia
sierolomucharatossuru 
---
loss:41 after 364200 iterations
---
 sclidyror
zmuluristra
p

---
 s
welonya
dynasaurus
xoria
nwamkordtyx
pshiimasiur
drkucerotops
benythrsiara
jangangosaurus
violophitax
tanophasaurus
auinteotor
xichandolodon
eurosauranis
lekvisaurus
alomimuanaralla
gigenichodosauru 
---
loss:41 after 367300 iterations
---
 ulophus
brychosaurus
argmosaurus
csyanomiseuravenwaurus
aumascaeratom
bospidaty
cakerthirotnosaurus
bikrbaus
wacrong
ziangy
dpheinnitoruurus
panetosaurus
asodonosaurus
alberodes
miorasaurus
kriplelong 
---
loss:41 after 367400 iterations
---
 
prigotosuchus
hiphophelao
bieisaurus
chiamidomuirisosaurus
gnagmus
cchongosaurus
eegkasaurus
drinosaurus
dangobohoertan
zhtiomicocousaurus
gliglus
cleptitangosaurus
dactyptor
ausahtonggjeleocronkus
e 
---
loss:41 after 367500 iterations
---
 pegyginjiopela
segtivus
chodontha
liantochua
hangosaurus
kuteokindongosampedosaurus
jengosaurus
kelusaurus
huiasanosauronhosaurus
gukoraplatosaurus
fus
crgnosaurus
lantosaurus
oparoto
adrosaurus
horni 
---
loss:41 after 367600 iterations
---
 vnosaurus
mugucburonosa

---
 olaatrta
teryptonosaurus
azupelsosaurus
adisaurus
ctihahidir
spolodomuatitasaururus
coeluctratodrus
cilandosaurus
baltiakus
xezanatos
eotosaurus
douperteor
zachicaroptor
cnagnaian
patosaurus
gongkinus 
---
loss:41 after 370700 iterations
---
 us
dypskioloptoysaurus
eitelteroenosaurus
dougambarasaurus
dipsolinak
aonosaurus
hibsanocerdosaurus
ducilis
muctonfeulus
famandysaurudus
acraphatosaurus
elbadodon
epmicaestinom
anhypentha
drephus
cron 
---
loss:41 after 370800 iterations
---
 
aratosaurus
jusaurusauius
ucrosaurus
gustrochusaususudelogis
harganisaurus
kualticen
takeveceratops
kerethoptor
brakokimang
heteralopshus
nyleisius
malannis
honestsauredogdolos
jentosaurus
hangopalon 
---
loss:41 after 370900 iterations
---
 hlosaurilonhus
lichyrakankaueuptanakorotitin
huhuanosaurus
laleoptortis
cronisaurus
mkomatolus
manteratosaurus
lemeeagdtaucolua
giallosaurus
egmarnosaurus
keguliphihus
magrimisaurus
miraceratus
argosa 
---
loss:41 after 371000 iterations
---
 opsnus
arameritostramos

---
 ostesius
hykotrinoquricon
zeiairotras
dolonindosaurus
huathestongosaurus
erosaurus
iarnosauriaphus
emponya
pelacondos
cenkhalobochosaurus
beinichesosaurus
chenosaulus
ephosaurus
fronikusaurus
ganytimo 
---
loss:41 after 374100 iterations
---
 us
jajurisalantridia
saatraphon
emomabancopety
datrilasaurus
luenaptarsaurus
gonotitanglushunosaurus
eusaurus
delodong
anahoiseuruus
jiausaurus
hlhwinyagis
tinacen
biaratsaurus
hullysaurus
hedanqucica 
---
loss:41 after 374200 iterations
---
 nosaurus
sulosaurus
ludloplodomanga
vanosainisudita
deosaurus
momesaurus
luceosaurus
latyrosaurus
kaptongephosaurus
oragstrassus
hidicansaurus
gurides
hxigusamphusaurus
pimusaurus
naptlbong
meuropanis 
---
loss:41 after 374300 iterations
---
 aurus
prganosaurus
paaisaurus
staliaerur
sauritosaurus
praapaptrannosaurosaurus
proeosaurus
notyrumnon
jwapcoracrovenator
saubenglosaurus
ooriwuus
beeaganasaurus
rampeptosaurus
tontiteodon
totanosaura 
---
loss:41 after 374400 iterations
---
 rus
sauravelesiomes
ojo

KeyboardInterrupt: 